# Create a Recipe

This Gen AI code creates personalized recipes based on ingredients available at home. Users input their available ingredients, and the code generates a tailored recipe while also utilizing DALL·E to produce a visual representation of the dish. Ideal for home cooks looking for creative meal ideas, this tool combines recipe generation with visual inspiration to enhance the cooking experience.

In [1]:
import os

In [22]:
import openai
from openai import OpenAI

In [23]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [15]:
def create_system_prompt():
    return "You're are helpful cooking assistant"

In [16]:
def create_dish_prompt(list_of_ingredient):
    prompt = f"Create a detailed recipe based on the following ingredient: {', '.join(list_of_ingredient)}.\n"\
            + f"Additionally, assign a title starting with 'Recipe Title: ' to this dish, which can be used to create a photorealistic image of it."
    return prompt

In [17]:
system_prompt = create_system_prompt()

In [19]:
recipe_prompt = create_dish_prompt(["Eggs", "Bacon", "Cheese"])
print(recipe_prompt)

Create a detailed recipe based on the following ingredient: Eggs, Bacon, Cheese.
Additionally, assign a title starting with 'Recipe Title: ' to this dish, which can be used to create a photorealistic image of it.


In [24]:
response = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages=[
                {"role":"system", "content":system_prompt},
                {"role":"user", "content":recipe_prompt},
            ]
)

In [40]:
result (response.choices[0].message.content)

Recipe Title: Cheesy Bacon and Egg Breakfast Casserole

Ingredients:
- 8 slices of bacon
- 6 large eggs
- 1 cup shredded cheddar cheese
- Salt and pepper to taste
- 1/2 cup milk
- 1/2 teaspoon garlic powder
- 1/2 teaspoon onion powder
- 1/4 teaspoon dried thyme
- Cooking spray

Instructions:
1. Preheat your oven to 350°F (175°C) and grease a 9x9 inch baking dish with cooking spray.
2. Cook the bacon in a skillet over medium heat until crispy. Remove from the pan and drain on paper towels. Once cooled, crumble the bacon into small pieces.
3. In a medium mixing bowl, whisk together the eggs, milk, garlic powder, onion powder, dried thyme, salt, and pepper until well combined.
4. Stir in 3/4 cup of the shredded cheddar cheese and half of the crumbled bacon.
5. Pour the egg mixture into the prepared baking dish and sprinkle the remaining bacon and cheese on top.
6. Bake in the preheated oven for 25-30 minutes or until the eggs are set and the cheese is melted and bubbly.
7. Remove from the

In [41]:
import re

In [42]:
def extract_title(recipe):
    return re.findall("^.*Recipe Title: .*$", recipe, re.MULTILINE)[0].strip().split("Recipe Title: ")[1]

In [43]:
recipe = response.choices[0].message.content

In [46]:
extract_title(recipe)

'Cheesy Bacon and Egg Breakfast Casserole'

In [47]:
import requests

In [48]:
import shutil

In [49]:
response_image = client.images.generate(
    
                    model = "dall-e-3",
                    prompt=extract_title(recipe),
                    size = "1024x1024",
                    n=1,
                    quality="standard"
                
)

In [55]:
image_url= response_image.data[0].url

In [57]:
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-Oe3dPFBB1ZiqizpTQTn9zAEQ/user-VV1AaQPu83fB6cNgYvWRadDC/img-d2hzg38Yeejg125FN8UPu1r9.png?st=2024-08-01T02%3A51%3A47Z&se=2024-08-01T04%3A51%3A47Z&sp=r&sv=2023-11-03&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-08-01T00%3A42%3A23Z&ske=2024-08-02T00%3A42%3A23Z&sks=b&skv=2023-11-03&sig=%2BJqQkEGX/CpJSrhhO3rPwrSKqnCPvTLz%2BQ%2BNoa2q6Ak%3D'

In [59]:
temp =requests.get(image_url)

In [62]:
def save_image(image_url, filename):
    
    image_res = requests.get(image_url, stream=True)
    
    if image_res.status_code==200:
        
        with open(filename, 'wb') as f:
            shutil.copyfileobj(image_res.raw, f)
    else:
        print('Error')
        
    return image_res.status_code 

In [63]:
save_image(image_url, "recipe.png")

200